In [36]:
# Dependencies

import pandas as pd
import matplotlib.pyplot as plt
import csv
import os

In [96]:
# Find path

Currentpath = os.getcwd()
Currentpath

'C:\\Users\\smanning\\Documents\\Project'

In [102]:
# Read CSV 2016 and 2017 CMS Enrollement file

Rawfile2017 = pd.read_csv("Enrollement.csv") 
Rawfile2016 = pd.read_csv('Medicare2016Enrollment.csv')

In [105]:
New_2017 = Rawfile2017[['FIPS State County Code', 'State', 'County', 'Enrollment']]
New_2016 = Rawfile2016[['FIPS State County Code', 'State', 'County', 'Enrollment']]
New_2016.head()

,FIPS State County Code,State,County,Enrollment
0,NaN,NaN,NaN,*
1,NaN,NaN,NaN,*
2,NaN,NaN,NaN,*
3,NaN,NaN,NaN,*
4,1001.0,AL,Autauga,*


In [110]:
# Clean Enrollement Dataframe. Remove '*' and Nan

New_CMS2017 = New_2017.dropna(how='any')
New_CMS2017 = New_CMS2017[Rawfile2017.Enrollment!= '*']
New_CMS2017['Enrollment'] = pd.to_numeric(New_CMS2017['Enrollment'])
New_CMS2017 = New_CMS2017.rename(columns = {'State': 'StateAbbrev'
                                ,'FIPS State County Code': 'FIPS'})

New_CMS2016 = New_2016.dropna(how='any')
New_CMS2016 = New_CMS2016[Rawfile2016.Enrollment!= '*']
New_CMS2016['Enrollment'] = pd.to_numeric(New_CMS2016['Enrollment'])
New_CMS2016 = New_CMS2016.rename(columns = {'State': 'StateAbbrev'
                                ,'FIPS State County Code': 'FIPS'})




C:\Users\smanning\python\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\smanning\python\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [112]:
# Groupy state and county

Cms_Enrollment_2017 = New_CMS2017.groupby(['FIPS', 'StateAbbrev', 'County'])
Final2017CMS = Cms_Enrollment_2017["Enrollment"].sum()

Cms_Enrollment_2016 = New_CMS2016.groupby(['FIPS', 'StateAbbrev', 'County'])
Final2016CMS = Cms_Enrollment_2016["Enrollment"].sum()

Final2016CMS.head()

# Put Cms_Enrollment in pd = Nick

FIPS    StateAbbrev  County 
1001.0  AL           Autauga     6672
1003.0  AL           Baldwin    33112
1005.0  AL           Barbour     4492
1007.0  AL           Bibb        3501
1009.0  AL           Blount      9425
Name: Enrollment, dtype: int64

In [113]:
# Read CSV population file

RawPopulation2017 = pd.read_csv("Population_Data.csv",encoding='iso-8859-1') 
RawPopulation2017.head()


,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,...,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,...,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,...,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,...,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,...,48,44,7,5,0,2,2,1,3,1


In [114]:
# Create New Dataframe

Population2017 = RawPopulation2017[['STNAME', 'CTYNAME', 'YEAR', 'AGEGRP','TOT_POP', 'TOT_MALE', 'TOT_FEMALE']]
Population2017.head()

,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE
0,Alabama,Autauga County,1,0,54571,26569,28002
1,Alabama,Autauga County,1,1,3579,1866,1713
2,Alabama,Autauga County,1,2,3991,2001,1990
3,Alabama,Autauga County,1,3,4290,2171,2119
4,Alabama,Autauga County,1,4,4290,2213,2077


In [115]:
CountyStateMap = pd.read_csv("CountyStateMap.csv") 
CountyStateMap.head()

,FIPS,StateName,StateAbbrev,CountyName,ShortCounty
0,1001.0,Alabama,AL,Autauga County,Autauga
1,1003.0,Alabama,AL,Baldwin County,Baldwin
2,1005.0,Alabama,AL,Barbour County,Barbour
3,1007.0,Alabama,AL,Bibb County,Bibb
4,1009.0,Alabama,AL,Blount County,Blount


In [ ]:
States = Final2017CMS['StateAbbrev']
Enrollment = Final2017CMS['Enrollment']

fig = gmaps.heatmap_layer(States, weights=Enrollment)

fig.add_layer(heat_layer)

fig

In [121]:
CombinedCMS1617 = pd.merge(Final2017CMS,Final2016CMS, on=['FIPS', 'StateAbbrev', 'County'], how='inner',suffixes=('_2017', '_2016'))
CombinedCMS1617.head()

,,,Enrollment_2017,Enrollment_2016
FIPS,StateAbbrev,County,,
1001.0,AL,Autauga,6961,6672
1003.0,AL,Baldwin,35220,33112
1005.0,AL,Barbour,4576,4492
1007.0,AL,Bibb,3550,3501
1009.0,AL,Blount,9647,9425


In [70]:
# Medicare vs Population (by state and by county)
# Attach FIPS dict = Sarah
# Compare 2016 and 2017 medicare enrollment
# Compare 2016 Medicare to inpatient data
# Compare Medicare enrollment in county to income (Use Census data) (higher medicare enrollement in count = higher income??)

